<a href="https://colab.research.google.com/github/hhauschild/keras-playground/blob/master/liveproject_dl_predict_2_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 create data frames


In [1]:
# create dataframes
import pandas as pd

teamurl = r"https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Teams.csv"
dfteams = pd.read_csv(teamurl, header=None, names=["ncaa_division","team"]);
print(dfteams.head())

gameurl = r"https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Games.csv"
dateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d")
#dfgames = pd.read_csv(gameurl, header = None, names=["date","home_team","home_score", "away_team","away_score" ], parse_dates=["date"], date_parser=dateparser, escapechar= "&");
dfgames = pd.read_csv(gameurl, header = None, names=["date","home_team","home_score", "away_team","away_score" ], parse_dates=["date"], dtype={"home_team":str,"home_score":int,"away_team":str,"away_score":int});
dfgames.head()

  ncaa_division         team
0  America East      Vermont
1  America East  Stony Brook
2  America East         UMBC
3  America East     Hartford
4  America East       Albany


,date,home_team,home_score,away_team,away_score
0,2015-11-13,Hawaii,87,Montana St.,76
1,2015-11-13,Eastern Mich.,70,Vermont,50
2,2015-11-13,Columbia,107,Kean,62
3,2015-11-13,La.-Monroe,88,McMurry,43
4,2015-11-13,Yale,70,Fairfield,57


2. Clean data: inspect team names

In [2]:
print(sorted(dfteams.team.unique()))
print(sorted(dfgames.home_team.unique()))
#print(sorted(dfgames.away_team.unique()))

['Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama State', 'Albany', 'Alcorn State', 'American University', 'Appalachian State', 'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State', 'Arkansas-Little Rock', 'Arkansas-Pine Bluff', 'Army', 'Auburn', 'Austin Peay', 'BYU', 'Ball State', 'Baylor', 'Belmont', 'Bethune-Cookman', 'Binghamton', 'Boise State', 'Boston College', 'Boston University', 'Bowling Green', 'Bradley', 'Brown', 'Bryant University', 'Bucknell', 'Buffalo', 'Butler', 'Cal Poly', 'Cal State Bakersfield', 'Cal State Fullerton', 'Cal State Northridge', 'California', 'California Baptist', 'Campbell', 'Canisius', 'Central Arkansas', 'Central Connecticut State', 'Central Michigan', 'Charleston', 'Charleston Southern', 'Charlotte', 'Chattanooga', 'Chicago State', 'Cincinnati', 'Citadel', 'Clemson', 'Cleveland State', 'Coastal Carolina', 'Colgate', 'Colorado', 'Colorado State', 'Columbia', 'Connecticut', 'Coppin State', 'Cornell', 'Creighton', 'Dartmo

2.1 clean HTML escaping


In [3]:
import html
html.unescape('A&amp;M-Corpus Chris')

'A&M-Corpus Chris'

In [4]:

dfgames.home_team=dfgames.home_team.apply(lambda x:html.unescape(x))
dfgames.away_team=dfgames.away_team.apply(lambda x:html.unescape(str(x)))

print(sorted(dfgames.home_team.unique()))
print(sorted(dfgames.away_team.unique()))

['A&M-Corpus Chris', 'Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama St.', 'Alabama State', 'Alas. Anchorage', 'Albany', 'Albany (NY)', 'Alcorn St.', 'Alcorn State', 'American', 'Appalachian St.', 'Arizona', 'Arizona St.', 'Ark.-Pine Bluff', 'Arkansas', 'Arkansas St.', 'Arkansas State', 'Army', 'Army West Point', 'Auburn', 'Austin Peay', 'Ave Maria', 'BYU', 'Bakersfield', 'Ball St.', 'Ball State', 'Baylor', 'Belmont', 'Bethune-Cookman', 'Binghamton', 'Boise St.', 'Boise State', 'Boston College', 'Boston U.', 'Bowling Green', 'Bradley', 'Brown', 'Bryant', 'Bucknell', 'Buffalo', 'Butler', 'CSU Bakersfield', 'CSU Fullerton', 'CSUN', 'Cal Baptist', 'Cal Poly', 'Cal St. Fullerton', 'Cal St. Northridge', 'California', 'Campbell', 'Canisius', 'Cent. Conn. St.', 'Centenary (LA)', 'Central Ark.', 'Central Conn. St.', 'Central Mich.', 'Chaminade', 'Charleston', 'Charleston So.', 'Charlotte', 'Chattanooga', 'Chicago St.', 'Chicago State', 'Cincinnati', 'Citadel', 'Cl

2.2 text normalzation

In [5]:
import re
import string
punctuation_table = str.maketrans('','',string.punctuation)
dfteams.team=dfteams.team.apply(lambda x:str(x).strip().translate(punctuation_table))
dfgames.home_team=dfgames.home_team.apply(lambda x:str(x).strip().translate(punctuation_table))
dfgames.away_team=dfgames.away_team.apply(lambda x:str(x).strip().translate(punctuation_table))

print(sorted(dfteams.team.unique()))
print(sorted(dfgames.home_team.unique()))
print(sorted(dfgames.away_team.unique()))


['Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama AM', 'Alabama State', 'Albany', 'Alcorn State', 'American University', 'Appalachian State', 'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State', 'ArkansasLittle Rock', 'ArkansasPine Bluff', 'Army', 'Auburn', 'Austin Peay', 'BYU', 'Ball State', 'Baylor', 'Belmont', 'BethuneCookman', 'Binghamton', 'Boise State', 'Boston College', 'Boston University', 'Bowling Green', 'Bradley', 'Brown', 'Bryant University', 'Bucknell', 'Buffalo', 'Butler', 'Cal Poly', 'Cal State Bakersfield', 'Cal State Fullerton', 'Cal State Northridge', 'California', 'California Baptist', 'Campbell', 'Canisius', 'Central Arkansas', 'Central Connecticut State', 'Central Michigan', 'Charleston', 'Charleston Southern', 'Charlotte', 'Chattanooga', 'Chicago State', 'Cincinnati', 'Citadel', 'Clemson', 'Cleveland State', 'Coastal Carolina', 'Colgate', 'Colorado', 'Colorado State', 'Columbia', 'Connecticut', 'Coppin State', 'Cornell', 'Creighton', 'Dartmouth'

3 add calculated columns


In [6]:
dfteams["home_points"] = dfteams.team.apply(lambda row: dfgames.query("home_team=='"+row+"'").home_score.sum())
dfteams["away_points"] = dfteams.team.apply(lambda row: dfgames.query("away_team=='"+row+"'").away_score.sum())
#dfteams["home_point_diff"] = dfteams.team.apply(lambda row: dfgames.query("home_team=='"+row+"'").apply(lambda x:x["home_score"]-x["away_score"]).sum())
dfteams

,ncaa_division,team,home_points,away_points
0,America East,Vermont,4991,5103
1,America East,Stony Brook,4560,4400
2,America East,UMBC,4971,4792
3,America East,Hartford,4377,4765
4,America East,Albany,1016,1134
...,...,...,...,...
348,Western Athletic,California Baptist,0,0
349,Western Athletic,Cal State Bakersfield,0,0
350,Western Athletic,Seattle,4079,2471
351,Western Athletic,UMKC,4719,4521


4 write csv

In [7]:
dfteams.to_csv('Games-Calculated.csv')